In [4]:
import dash
from dash import Dash, html, dash_table
import plotly.express as px
import pandas as pd

# Load the data
df = pd.read_csv('gyroscope_readings.csv')

app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("Gyroscope Data Visualization"),

    # Dropdown to select graph type
    html.Label("Select Graph Type:"),
    dcc.Dropdown(
        id='graph-type',
        options=[
            {'label': 'Scatter Plot', 'value': 'scatter'},
            {'label': 'Line Chart', 'value': 'line'},
            {'label': 'Distribution Plot', 'value': 'histogram'}
        ],
        value='scatter'
    ),

    # Dropdown to select data variables
    html.Label("Select Variables:"),
    dcc.Dropdown(
        id='variables',
        options=[
            {'label': 'X', 'value': 'x'},
            {'label': 'Y', 'value': 'y'},
            {'label': 'Z', 'value': 'z'}
        ],
        value=['x', 'y', 'z'],
        multi=True
    ),

    # Input for number of data samples
    html.Label("Number of Samples:"),
    dcc.Input(id='num-samples', type='number', value=10, min=1),

    # Buttons for next/previous navigation
    html.Button('Previous', id='prev-button', n_clicks=0),
    html.Button('Next', id='next-button', n_clicks=0),

    # Graph to display data
    dcc.Graph(id='graph'),

    # Data summary table
    dash_table.DataTable(id='data-summary', columns=[
        {"name": i, "id": i} for i in ['Variable', 'Mean', 'Min', 'Max', 'Standard Deviation']
    ])
])

# Callback to update graph and summary based on user input
@app.callback(
    [Output('graph', 'figure'),
     Output('data-summary', 'data')],
    [Input('graph-type', 'value'),
     Input('variables', 'value'),
     Input('num-samples', 'value'),
     Input('prev-button', 'n_clicks'),
     Input('next-button', 'n_clicks')],
    [State('num-samples', 'value')]
)
def update_graph(graph_type, variables, num_samples, prev_clicks, next_clicks, current_samples):
    start = max((next_clicks - prev_clicks) * num_samples, 0)
    end = start + num_samples
    filtered_df = df.iloc[start:end]

    if graph_type == 'scatter':
        fig = px.scatter(filtered_df, x='timestamp', y=variables, title='Scatter Plot')
    elif graph_type == 'line':
        fig = px.line(filtered_df, x='timestamp', y=variables, title='Line Chart')
    elif graph_type == 'histogram':
        fig = px.histogram(filtered_df, x=variables, title='Distribution Plot')

    summary_data = []
    for var in variables:
        summary_data.append({
            'Variable': var,
            'Mean': filtered_df[var].mean(),
            'Min': filtered_df[var].min(),
            'Max': filtered_df[var].max(),
            'Standard Deviation': filtered_df[var].std()
        })

    return fig, summary_data

if __name__ == '__main__':
    app.run_server(debug=True)
